In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler, RobustScaler, StandardScaler
import time

import tensorflow as tf
tf.random.set_seed(42)

def mape(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / max(y_true))) * 100

def RMSELoss(yhat,y):
    return torch.sqrt(torch.mean((yhat-y)**2))

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [2]:
# Specifying the path of file
path= r"C:\Users\sashah8\OneDrive - North Carolina State University\Desktop\Q4\Data\L4\data_cleaned"

# Listing just the required files
fileList = os.listdir(path)
fileList.remove("Date.csv")

# Previewing the required file names
print(fileList)

['Capital.csv', 'Central.csv', 'Dunwoody.csv', 'Genese.csv', 'Hudson.csv', 'LongIsland.csv', 'Milwood.csv', 'NYC.csv']


In [3]:
# Just for previewing the columns
pd.read_csv(os.path.join(path, fileList[7]))

,Demand,Normalized_net,Temprature
0,5272.0,0.485856,-5.8
1,5092.6,0.469323,-6.1
2,4909.5,0.452449,-7.5
3,4771.4,0.439722,-8.4
4,4698.5,0.433003,-7.4
...,...,...,...
43819,5909.3,0.544588,3.4
43820,5689.2,0.524304,3.3
43821,5488.3,0.505790,2.6
43822,5276.0,0.486225,2.7


### PROCESSING THE DATA

In [4]:
# Restricting the load data to a single zone
fileName  = fileList[7]

zoneData = pd.read_csv(os.path.join(path, fileName), usecols= ["Demand", "Temprature"])
zoneData.columns = ["Load_"+str(fileName[:-4]), "Temp_"+str(fileName[:-4])]

# For fixing one missing row
# zoneData = pd.concat([zoneData.loc[[0]], zoneData], axis= 0).reset_index(drop= True)

zoneData

,Load_NYC,Temp_NYC
0,5272.0,-5.8
1,5092.6,-6.1
2,4909.5,-7.5
3,4771.4,-8.4
4,4698.5,-7.4
...,...,...
43819,5909.3,3.4
43820,5689.2,3.3
43821,5488.3,2.6
43822,5276.0,2.7


### FEATURE ENGINEERING AND SPLITTING INTO TRAIN-TEST SPLIT

In [5]:
def lagGenerator(dF, lagList):
    dF_temp = dF.filter(regex= "Load").copy()
    
    for i in lagList:
        dF_temp.loc[:, "Lag-" + str(i)] = dF_temp.iloc[:,[0]].shift(i*24)
    
    return(dF_temp.filter(regex= "Lag"))

In [6]:
Load_lags = lagGenerator(dF=zoneData, lagList=[2,7])
Load_lags.head(2)

,Lag-2,Lag-7
0,NaN,NaN
1,NaN,NaN


In [7]:
dataDf = pd.concat([zoneData, Load_lags], axis= 1).dropna(axis= 0).reset_index(drop= True)
dataDf

,Load_NYC,Temp_NYC,Lag-2,Lag-7
0,5715.8,-13.9,5194.9,5272.0
1,5461.3,-14.2,4967.4,5092.6
2,5320.8,-14.8,4826.9,4909.5
3,5261.7,-15.6,4781.3,4771.4
4,5292.5,-16.0,4816.2,4698.5
...,...,...,...,...
43651,5909.3,3.4,5649.6,5756.6
43652,5689.2,3.3,5563.1,5594.8
43653,5488.3,2.6,5430.6,5431.0
43654,5276.0,2.7,5226.6,5244.8


In [8]:
# Setting the train-test split ratio
ratio = 0.1

In [9]:
Num_train, Num_test = int(len(dataDf)/24 * (1 - ratio)) * 24, int(len(dataDf)/24 * ratio) * 24

In [10]:
# Splitting the data into train and test
data_train, data_test = dataDf[:Num_train], dataDf[Num_train:]

In [11]:
Num_train, Num_valid = int(len(data_train)/24 * (1 - ratio)) * 24, int(len(data_train)/24 * ratio) * 24

In [12]:
# Splitting the data into train and test
data_train, data_valid = data_train[:Num_train], data_train[Num_train:]

### SPLITTING INTO X AND Y PART WITH SEQUENCE BATCHING

In [13]:
X_train_df, Y_train_df = data_train.iloc[:, 1:], data_train.iloc[:, :1]

X_valid_df, Y_valid_df = data_valid.iloc[:, 1:], data_valid.iloc[:, :1]

X_test_df, Y_test_df = data_test.iloc[:, 1:], data_test.iloc[:, :1]

In [14]:
# Preparing the scaler
ValScaler = MinMaxScaler()

X_train = ValScaler.fit_transform(X_train_df)
X_valid = ValScaler.transform(X_valid_df)
X_test = ValScaler.transform(X_test_df)

Y_train = ValScaler.fit_transform(Y_train_df)
Y_valid = ValScaler.transform(Y_valid_df)
Y_test = ValScaler.transform(Y_test_df)

In [15]:
def splitData(dataset):
    data = np.array(np.split(dataset, len(dataset)/24))
    return data

In [16]:
def supervisedSet(X_part, Y_part, steps= 24):
    X_array, Y_array = np.array(X_part), np.array(Y_part)
    X, Y = [], []
    
    for i in range(len(X_array)):
        endIdx = i + steps
        
        if endIdx > len(X_array):
            break
        
        batch_X, batch_Y = X_array[i:endIdx], Y_array[i:endIdx]
        
        X.append(batch_X)
        Y.append(batch_Y)
        
    return np.array(X), np.array(Y)

In [17]:
X_train, Y_train = supervisedSet(X_train, Y_train)

X_valid, Y_valid = splitData(X_valid), splitData(Y_valid)

X_test, Y_test = splitData(X_test), splitData(Y_test)

### WORKING ON THE MODEL

In [18]:
epochs = 100
batchSize = 32

In [19]:
# Encoder-decoder architecture [tanh required to use tensorflow on gpu]
model = tf.keras.Sequential([
    tf.keras.layers.LSTM(units= 128, activation= "tanh", input_shape= (X_train.shape[1], X_train.shape[2]), dropout= 0.2),
    tf.keras.layers.RepeatVector(Y_train.shape[1]),
    tf.keras.layers.LSTM(units= 128, activation= "tanh", return_sequences= True, dropout= 0.2),
    tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(units= 128, activation= "relu")),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(units= 1))
])

model.compile(loss= "mse", optimizer= "adam",  metrics= ['mape', 'mae'])

In [20]:
# model = tf.keras.Sequential([
#     tf.keras.layers.LSTM(units= 256, activation= "tanh", input_shape= (X_train.shape[1], X_train.shape[2])),
#     tf.keras.layers.Dense(units= 128, activation= "tanh"),
#     tf.keras.layers.Dense(units= 1)
# ])

# model.compile(loss= "mse", optimizer= "adam",  metrics= ['mape', 'mae'])

In [21]:
# Storing the start time of model fitting process
startTime = time.time()

In [22]:
model.fit(X_train, Y_train, epochs= epochs, batch_size= batchSize, verbose= 1, shuffle= True, validation_data= (X_valid, Y_valid))

Epoch 1/100
1105/1105 [==============================] - 25s 19ms/step - loss: 0.0125 - mape: 5010.8481 - mae: 0.0855 - val_loss: 0.0058 - val_mape: 50.5975 - val_mae: 0.0610
Epoch 2/100
1105/1105 [==============================] - 19s 17ms/step - loss: 0.0080 - mape: 4696.0420 - mae: 0.0676 - val_loss: 0.0076 - val_mape: 51.0620 - val_mae: 0.0653
Epoch 3/100
1105/1105 [==============================] - 19s 17ms/step - loss: 0.0058 - mape: 4013.9927 - mae: 0.0576 - val_loss: 0.0090 - val_mape: 47.0163 - val_mae: 0.0671
Epoch 4/100
1105/1105 [==============================] - 18s 17ms/step - loss: 0.0053 - mape: 4209.2202 - mae: 0.0548 - val_loss: 0.0080 - val_mape: 47.3732 - val_mae: 0.0637
Epoch 5/100
1105/1105 [==============================] - 18s 16ms/step - loss: 0.0048 - mape: 4024.8005 - mae: 0.0520 - val_loss: 0.0099 - val_mape: 43.0940 - val_mae: 0.0705
Epoch 6/100
1105/1105 [==============================] - 18s 16ms/step - loss: 0.0045 - mape: 3829.5522 - mae: 0.0498 - val_l

1105/1105 [==============================] - 19s 18ms/step - loss: 0.0020 - mape: 2565.8296 - mae: 0.0333 - val_loss: 0.0078 - val_mape: 38.0845 - val_mae: 0.0586
Epoch 48/100
1105/1105 [==============================] - 20s 18ms/step - loss: 0.0021 - mape: 3021.9573 - mae: 0.0335 - val_loss: 0.0080 - val_mape: 37.0233 - val_mae: 0.0588
Epoch 49/100
1105/1105 [==============================] - 19s 17ms/step - loss: 0.0020 - mape: 2534.1599 - mae: 0.0330 - val_loss: 0.0078 - val_mape: 39.4168 - val_mae: 0.0578
Epoch 50/100
1105/1105 [==============================] - 19s 17ms/step - loss: 0.0019 - mape: 2642.2864 - mae: 0.0326 - val_loss: 0.0078 - val_mape: 39.2801 - val_mae: 0.0587
Epoch 51/100
1105/1105 [==============================] - 19s 17ms/step - loss: 0.0019 - mape: 2623.6799 - mae: 0.0323 - val_loss: 0.0079 - val_mape: 42.7319 - val_mae: 0.0590
Epoch 52/100
1105/1105 [==============================] - 18s 16ms/step - loss: 0.0019 - mape: 2627.8906 - mae: 0.0323 - val_loss: 0.

1105/1105 [==============================] - 15s 14ms/step - loss: 0.0011 - mape: 3049.0110 - mae: 0.0248 - val_loss: 0.0083 - val_mape: 46.5716 - val_mae: 0.0636
Epoch 94/100
1105/1105 [==============================] - 15s 14ms/step - loss: 0.0011 - mape: 2669.8904 - mae: 0.0248 - val_loss: 0.0085 - val_mape: 46.0209 - val_mae: 0.0639
Epoch 95/100
1105/1105 [==============================] - 15s 14ms/step - loss: 0.0010 - mape: 2883.0830 - mae: 0.0242 - val_loss: 0.0088 - val_mape: 46.5222 - val_mae: 0.0663
Epoch 96/100
1105/1105 [==============================] - 15s 14ms/step - loss: 0.0011 - mape: 3030.6406 - mae: 0.0244 - val_loss: 0.0087 - val_mape: 46.6338 - val_mae: 0.0648
Epoch 97/100
1105/1105 [==============================] - 15s 14ms/step - loss: 0.0011 - mape: 3061.6926 - mae: 0.0246 - val_loss: 0.0090 - val_mape: 47.7488 - val_mae: 0.0667
Epoch 98/100
1105/1105 [==============================] - 15s 14ms/step - loss: 0.0010 - mape: 2918.4634 - mae: 0.0240 - val_loss: 0.

In [23]:
calcExecTime = (time.time() - startTime)
print('Total execution time in seconds: ' + str(calcExecTime))

Total execution time in seconds: 1731.761873960495


### MAKING PREDICTIONS AND GENERATING RESULTS

In [24]:
predictions = model.predict(X_test)

6/6 [==============================] - 1s 9ms/step


In [25]:
predArr = predictions.reshape(predictions.shape[0] * predictions.shape[1], predictions.shape[2]).ravel()
predArr

array([0.19205846, 0.17388847, 0.15608777, ..., 0.31883657, 0.27348608,
       0.23650986], dtype=float32)

In [26]:
Y_testArr = Y_test.reshape(Y_test.shape[0] * Y_test.shape[1], Y_test.shape[2]).ravel()
Y_testArr

array([0.45135175, 0.41929074, 0.39828295, ..., 0.28152087, 0.25307728,
       0.2245935 ])

In [27]:
def inverse_transform(df):
    column_names = df.columns
    for col in column_names:
        df[col] = ValScaler.inverse_transform(df[[col]]) 
    return df

In [28]:
comparisionDf = pd.DataFrame(data= {"Values": Y_testArr, "Predictions": predArr})
comparisionDf = inverse_transform(comparisionDf)
comparisionDf

,Values,Predictions
0,6755.9,4820.562012
1,6516.6,4684.942871
2,6359.8,4552.080078
3,6324.5,4459.239258
4,6495.6,4483.193359
...,...,...
4363,5909.3,6182.153809
4364,5689.2,6006.617188
4365,5488.3,5766.820312
4366,5276.0,5428.329102


In [29]:
mape(comparisionDf["Values"], comparisionDf["Predictions"])

8.32387868945907